In [17]:
#!pip install tensorflow_text==2.5.0.
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from scipy.stats import mode
import re
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.model_selection import KFold

In [23]:
twitter = pd.read_csv("labeled_tweet_table.csv")
twitter = twitter[twitter["Race"].notna()]
twitter = twitter[twitter["Race"] != 5]
twitter["Race"] -= 1
twitter["Race"].value_counts()

3.0    242886
0.0     28720
1.0     17797
2.0      9885
Name: Race, dtype: int64

In [24]:
regexMap={r"<[\w'/'\s]*>": "",r"[\'\"\-]+": "",r"@[\w]+":"",r"http\S+" : ""}
def preprocess(datainput):
    t=datainput
    for regx in regexMap.keys():
        try:
            t = re.sub(regx, regexMap[regx], t)
        except TypeError:
            continue
    return t
twitter["Tweet"] = twitter["Tweet"].apply(preprocess)
twitter

,Tweet,Name,Screen Name,Description,Lang,img_path,Race,Under 21
0,"YKAR, a futuristic sans serif font by #Freeb...",Chad Boyce,djsnipa1,"Multimedia Developer, Graphic Designer, DJ, an...",NaN,profile pics/60147.jpeg,3.0,0
1,Who can I contact about the very rude and poo...,Chad Boyce,djsnipa1,"Multimedia Developer, Graphic Designer, DJ, an...",NaN,profile pics/60147.jpeg,3.0,0
2,I’d like to win!,Chad Boyce,djsnipa1,"Multimedia Developer, Graphic Designer, DJ, an...",NaN,profile pics/60147.jpeg,3.0,0
3,,Chad Boyce,djsnipa1,"Multimedia Developer, Graphic Designer, DJ, an...",NaN,profile pics/60147.jpeg,3.0,0
4,Now Im heading to B1000th Floor! #quickrogue,Chad Boyce,djsnipa1,"Multimedia Developer, Graphic Designer, DJ, an...",NaN,profile pics/60147.jpeg,3.0,0
...,...,...,...,...,...,...,...,...
313398,Im raising money for Help Feed the Homeless. C...,Joseph Aragunde,thomasaragu,General Contractor plus construction worker go...,NaN,profile pics/64221.jpeg,3.0,0
313399,Im raising money for Help Feed the Homeless. C...,Joseph Aragunde,thomasaragu,General Contractor plus construction worker go...,NaN,profile pics/64221.jpeg,3.0,0
313400,Im raising money for Help Feed the Homeless. C...,Joseph Aragunde,thomasaragu,General Contractor plus construction worker go...,NaN,profile pics/64221.jpeg,3.0,0
313401,Im raising money for Help Feed the Homeless. C...,Joseph Aragunde,thomasaragu,General Contractor plus construction worker go...,NaN,profile pics/64221.jpeg,3.0,0


In [25]:
twitter['Race'].value_counts()

3.0    242886
0.0     28720
1.0     17797
2.0      9885
Name: Race, dtype: int64

In [26]:
names = twitter["Name"].unique()
labels = []
tweet_dict = {}
for name in names:
    tweet_dict[name] = twitter[twitter["Name"] == name]["Tweet"].tolist()

In [27]:
label_dict = twitter.set_index("Name").to_dict()["Race"]

In [28]:
concatenated_tweets = []
labels = []
for name in names:
    concat = ''.join(str(tweet_dict[name]))
    concatenated_tweets.append(concat)
    labels.append(label_dict[name])

concat_df = pd.DataFrame({"User": np.array(names), "Tweet": concatenated_tweets, "Race": np.array(labels)})
len(names), len(concatenated_tweets), len(labels)

(3056, 3056, 3056)

In [29]:
concat_df["Race"].value_counts()

3.0    2475
0.0     298
1.0     181
2.0     102
Name: Race, dtype: int64

In [30]:
x = concat_df
y = concat_df['Race']

In [31]:
x.drop('Race',inplace=True,axis=1)
x

,User,Tweet
0,Chad Boyce,"['YKAR, a futuristic sans serif font by #Fre..."
1,Tomato 😷,"['In other words, it’s good news about the vac..."
2,parker,"[' ❤️ 🙏', ' bonk', ' #FFXIV400kSweepstakes', ..."
3,Kevin Jones,[' How about pizza dipped in water 🤦🏻\u200d♂️'...
4,Joe Duhamel,[' \n1) Hire better programmers\n2) Your webs...
...,...,...
3051,Shortie,['. on #Periscope: Im back someone called hung...
3052,Ben,['']
3053,Meagan Lovely,['Womens Guide to Burn Fat and Build Muscle: T...
3054,Megan Schuitema,[' Even though school is cancelled and grades ...


In [35]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

ros = RandomOverSampler(random_state=42)

x_ros, y_ros = ros.fit_resample(x, y)

print('Original dataset shape', Counter(y))
print('Resample dataset shape', Counter(y_ros))

Original dataset shape Counter({3.0: 2475, 0.0: 298, 1.0: 181, 2.0: 102})
Resample dataset shape Counter({3.0: 2475, 2.0: 2475, 0.0: 2475, 1.0: 2475})


In [36]:
df = x_ros
df['Race'] = y_ros
df

,User,Tweet,Race
0,Chad Boyce,"['YKAR, a futuristic sans serif font by #Fre...",3.0
1,Tomato 😷,"['In other words, it’s good news about the vac...",3.0
2,parker,"[' ❤️ 🙏', ' bonk', ' #FFXIV400kSweepstakes', ...",3.0
3,Kevin Jones,[' How about pizza dipped in water 🤦🏻\u200d♂️'...,3.0
4,Joe Duhamel,[' \n1) Hire better programmers\n2) Your webs...,3.0
...,...,...,...
9895,Abhishek Chaliha,['The four cylinder WagonR makes a comeback in...,2.0
9896,Devendra,"[' I think you mean... ', ' Bruh I deleted al...",2.0
9897,díẹ̀kọláàánútímorígba,['Ọmọ this album that Adekunle Gold is cooking...,2.0
9898,jun 💌,"['*grapples the baki* ', 'DOES ANYONE HAVE THA...",2.0


In [18]:
def constructModel():
    bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
    bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
    text_input = keras.layers.Input(shape=(), dtype=tf.string)
    preprocessed_text = bert_preprocess(text_input)
    outputs = bert_encoder(preprocessed_text)

    dense = keras.layers.Dense(4, activation='softmax')(outputs["pooled_output"])

    model = keras.Model(inputs=[text_input], outputs=[dense])
    model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    model.summary()
    return model

model = constructModel()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_word_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'encoder_outputs':  109482241   keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [37]:
accs = []
reports = []
confusion_matrices = []

kf = KFold(n_splits = 5)
i = 1
for train, test in kf.split(concat_df):
    print("Fold %d:" % i)
    print()
    i += 1
    train_df = df.iloc[train]
    test_df = df.iloc[test]
    
    x_train, y_train = train_df["Tweet"], train_df["Race"]
    x_test, y_test = test_df["Tweet"], test_df["Race"]
    
    model.fit(x_train, y_train, epochs = 5)
    y_pred = np.argmax(model.predict(x_test), axis=1)
    acc = accuracy_score(y_test, y_pred)
    accs.append(acc)
    print("Accuracy: ", acc)
    print()
    report = classification_report(y_test, y_pred)
    print(report)
    print()
    reports.append(report)
    cm = confusion_matrix(y_test, y_pred, normalize='true')
    print(cm)
    print()
    confusion_matrices.append(cm)

Fold 1:

Epoch 1/5
77/77 [==============================] - 23s 222ms/step - loss: 0.7091 - accuracy: 0.8011
Epoch 2/5
77/77 [==============================] - 17s 221ms/step - loss: 0.6904 - accuracy: 0.8044
Epoch 3/5
77/77 [==============================] - 17s 220ms/step - loss: 0.6785 - accuracy: 0.8044
Epoch 4/5
77/77 [==============================] - 17s 220ms/step - loss: 0.6815 - accuracy: 0.8044
Epoch 5/5
77/77 [==============================] - 17s 219ms/step - loss: 0.6717 - accuracy: 0.8040
Accuracy:  0.8316993464052288

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        52
         1.0       0.00      0.00      0.00        36
         2.0       0.00      0.00      0.00        15
         3.0       0.83      1.00      0.91       509

    accuracy                           0.83       612
   macro avg       0.21      0.25      0.23       612
weighted avg       0.69      0.83      0.76       612


[[0. 0. 0. 1.]
 [0. 0. 0

/share/pkg.7/python3/3.8.10/install/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/share/pkg.7/python3/3.8.10/install/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/share/pkg.7/python3/3.8.10/install/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

77/77 [==============================] - 17s 219ms/step - loss: 0.6477 - accuracy: 0.8164
Epoch 2/5
77/77 [==============================] - 17s 220ms/step - loss: 0.6507 - accuracy: 0.8155
Epoch 3/5
77/77 [==============================] - 17s 221ms/step - loss: 0.6602 - accuracy: 0.8147
Epoch 4/5
77/77 [==============================] - 17s 220ms/step - loss: 0.6344 - accuracy: 0.8160
Epoch 5/5
77/77 [==============================] - 17s 221ms/step - loss: 0.6406 - accuracy: 0.8164
Accuracy:  0.7823240589198036

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        67
         1.0       0.00      0.00      0.00        45
         2.0       0.00      0.00      0.00        21
         3.0       0.78      1.00      0.88       478

    accuracy                           0.78       611
   macro avg       0.20      0.25      0.22       611
weighted avg       0.61      0.78      0.69       611


[[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.

/share/pkg.7/python3/3.8.10/install/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/share/pkg.7/python3/3.8.10/install/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/share/pkg.7/python3/3.8.10/install/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

77/77 [==============================] - 17s 222ms/step - loss: 0.6536 - accuracy: 0.8098
Epoch 2/5
77/77 [==============================] - 17s 222ms/step - loss: 0.6582 - accuracy: 0.8098
Epoch 3/5
77/77 [==============================] - 17s 220ms/step - loss: 0.6542 - accuracy: 0.8098
Epoch 4/5
77/77 [==============================] - 17s 221ms/step - loss: 0.6512 - accuracy: 0.8094
Epoch 5/5
77/77 [==============================] - 17s 219ms/step - loss: 0.6480 - accuracy: 0.8106
Accuracy:  0.8117839607201309

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        59
         1.0       0.00      0.00      0.00        33
         2.0       0.00      0.00      0.00        23
         3.0       0.81      1.00      0.90       496

    accuracy                           0.81       611
   macro avg       0.20      0.25      0.22       611
weighted avg       0.66      0.81      0.73       611


[[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.

/share/pkg.7/python3/3.8.10/install/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/share/pkg.7/python3/3.8.10/install/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/share/pkg.7/python3/3.8.10/install/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

77/77 [==============================] - 17s 221ms/step - loss: 0.6472 - accuracy: 0.8127
Epoch 2/5
77/77 [==============================] - 17s 221ms/step - loss: 0.6413 - accuracy: 0.8123
Epoch 3/5
77/77 [==============================] - 17s 220ms/step - loss: 0.6397 - accuracy: 0.8131
Epoch 4/5
77/77 [==============================] - 17s 219ms/step - loss: 0.6547 - accuracy: 0.8131
Epoch 5/5
77/77 [==============================] - 17s 217ms/step - loss: 0.6433 - accuracy: 0.8127
Accuracy:  0.8003273322422259

              precision    recall  f1-score   support

         0.0       1.00      0.01      0.03        70
         1.0       0.00      0.00      0.00        27
         2.0       0.00      0.00      0.00        26
         3.0       0.80      1.00      0.89       488

    accuracy                           0.80       611
   macro avg       0.45      0.25      0.23       611
weighted avg       0.75      0.80      0.71       611


[[0.01428571 0.         0.         0.985714

/share/pkg.7/python3/3.8.10/install/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/share/pkg.7/python3/3.8.10/install/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/share/pkg.7/python3/3.8.10/install/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

77/77 [==============================] - 17s 218ms/step - loss: 0.6472 - accuracy: 0.8061
Epoch 2/5
77/77 [==============================] - 17s 217ms/step - loss: 0.6499 - accuracy: 0.8074
Epoch 3/5
77/77 [==============================] - 17s 218ms/step - loss: 0.6485 - accuracy: 0.8070
Epoch 4/5
77/77 [==============================] - 17s 218ms/step - loss: 0.6452 - accuracy: 0.8057
Epoch 5/5
77/77 [==============================] - 17s 217ms/step - loss: 0.6499 - accuracy: 0.8082
Accuracy:  0.8199672667757774

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        50
         1.0       0.00      0.00      0.00        40
         2.0       0.00      0.00      0.00        17
         3.0       0.82      0.99      0.90       504

    accuracy                           0.82       611
   macro avg       0.21      0.25      0.23       611
weighted avg       0.68      0.82      0.74       611


[[0.         0.         0.         1.      

/share/pkg.7/python3/3.8.10/install/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/share/pkg.7/python3/3.8.10/install/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/share/pkg.7/python3/3.8.10/install/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [38]:
np.average(accs)

0.8092203930126333

In [39]:
sum(confusion_matrices) / 5

array([[2.85714286e-03, 0.00000000e+00, 0.00000000e+00, 9.97142857e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [7.93650794e-04, 0.00000000e+00, 3.96825397e-04, 9.98809524e-01]])